# Пример использования

#### __```Импорт библиотек```__
---

In [ ]:
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from torchvision import transforms

from torchsummary import summary

#### __```Указание путей к расположению необработанных данных и расположению нового датасета```__
---

In [ ]:
if os.getcwd() == '/content':
    from google.colab import drive
    drive.mount('/content/drive'); 
    
    # Путь к необработанным данным на диске
    data_directory = '/content/drive/MyDrive/FMCW_Radar'; 

else: data_directory = str(input('Задайте путь к данным с радара: '));

#### __```Импорт набора инструментов```__
---

In [ ]:
!git clone https://github.com/imaks79/assets.git

In [ ]:
from assets.nn.utils import torch_stats, torch_seed, imshow_predicts, imshow
from assets.nn.processing import train_processing, predict_processing, accuracy, evaluate

from assets.models.resnet import Resnet18_modified

from assets.dataset.raw_to_npy import convert_data, get_class_names
from assets.dataset.dataloader import MyLoader
from assets.dataset.utils import dataset_paths_create

#### __```Указание путей к расположению необработанных данных и расположению нового датасета```__
---

In [ ]:
data_assignment = 'train';
raw_data_path = os.path.join(data_directory, 'rawdata');
dataset_directory = os.path.join(data_directory, 'adcdata');

#### __```Конвертировать .bin файлы в .npy```__
---

In [ ]:
dataset_directory_df = convert_data(raw_data_path, dataset_directory, data_assignment);

#### __```Выбор устройства torch```__
---

In [ ]:
device, dtype, num_workers = torch_stats();
torch_seed(seed = 42, deterministic = True);

#### __```Создание набора преобразований```__
---

In [ ]:
train_transform =  transforms.Compose([
        transforms.ToTensor(),
    ]);

#### __```Создание датасета из доступных файлов```__
---

In [ ]:
VALID_RATIO = 0.8;
train_data_path = os.path.join(dataset_directory, 'train');
test_data_path = os.path.join(dataset_directory, 'test');

data_paths, class_to_idx, idx_to_class = dataset_paths_create(train_data_path, test_data_path, VALID_RATIO);

In [ ]:
train_dataset = MyLoader(data_paths['train'], class_to_idx, train_transform);
valid_dataset = MyLoader(data_paths['valid'], class_to_idx, train_transform);


class_names = get_class_names(idx_to_class);

print(f'Доступные классы: {class_names}');
print(f'Число тренеровочных выборок: {len(train_dataset)}');
print(f'Число вилидационных выборок: {len(valid_dataset)}');
try: print('Число тестовых выборок:', {len(data_paths['test'])});
except: print('Тестовая выборка отсутсвует.');

#### __```Создание загрузчиков данных```__
---

In [ ]:
BATCH_SIZE = 5;

if torch.cuda.is_available(): kwarg = {'generator': torch.Generator(device = 'cuda')};
else: kwarg = {'num_workers': min(BATCH_SIZE, num_workers)};

train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, **kwarg);
valid_loader = DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle = True, **kwarg);
try: test_loader = DataLoader(data_paths['test'], batch_size = BATCH_SIZE, shuffle = False, **kwarg);
except: print('Тестовая выборка отсутсвует.');

#### __```Просмотреть созданный набор```__
---

In [ ]:
imshow((next(iter(train_loader)))[0], 1, device);

#### __```Создание модели НС```__
---

In [ ]:
model = Resnet18_modified(input_channels = 4, n_classes = len(class_names)).to(device);

#### __```Создание параметров обучения НС```__
---

In [ ]:
LR = 0.001;

criterion = nn.CrossEntropyLoss().to(device);
trainable_parameters = filter(lambda p: p.requires_grad, model.parameters());
optimizer = optim.SGD(trainable_parameters, lr = LR, momentum = 0.9);

#### __```Проверить обученность сети на начальном этапе```__
---

In [ ]:
best_epoch = 0;
test_loss, test_acc = evaluate(model, valid_loader, criterion, accuracy, device);
print(f'best epoch {best_epoch}: Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%');

#### __```Начать обучение сети```__
---

In [ ]:
EPOCHS = 20;
train_processing(EPOCHS, model, train_loader, valid_loader, optimizer, criterion, device);

#### __```Проверить лучшую эпоху```__
---

In [ ]:
model.load_state_dict(torch.load('best_model.pt'));
test_loss, test_acc = evaluate(model, valid_loader, criterion, accuracy, device);
print(10 * '--', f'\nbest epoch {best_epoch}: Test Loss: {test_loss:.3f} | Test Acc: {test_acc * 100:.2f}%');

#### __```Начать тестирование сети```__
---

In [ ]:
try: data_to_imshow, predicts, truth, df = predict_processing(model, test_loader, device, class_names);
except: print('Тестовая выборка отсутсвует.');

#### __```Просмотреть результат тестирования сети```__
---

In [ ]:
num_batches_to_show = 1;
try: imshow_predicts(data_to_imshow, BATCH_SIZE, num_batches_to_show, predicts, truth, device, class_names);
except: print('Тестовая выборка отсутсвует.');